In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://oraprdnt.uqtr.uquebec.ca/pls/apex/f?p=106:10::::10:P10_CD_PGM,P10_RECH_CRITERE,P10_RECH_VALEUR,P10_RECH_DESC:7947,P2_DOMAINE_ETUDE,EI,%5CGénie%5C&domaine-genie"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [8]:
page_soup = soup(driver.page_source, 'lxml')

In [12]:
tables = page_soup.find("div", {"class": "listeCours"}).findAll("table", {"class": "courspgmhoraire"})
len(tables)

4

In [15]:
link_texts = [container.find("a").text.strip() for table in tables for container in table.findAll("td", {"class": "haut lienSigleCours"})]
len(link_texts)

41

In [16]:
link_texts

['GEI1085',
 'GEI1086',
 'GIA1047',
 'GIA1058',
 'GIA1077',
 'GMC1006',
 'GMC1016',
 'GMC1017',
 'GMC1024',
 'GMC1025',
 'GMC1028',
 'GMC1031',
 'GMC1032',
 'GMC1034',
 'GMC1035',
 'GMC1036',
 'GMC1037',
 'GMC1053',
 'ING1039',
 'ING1040',
 'ING1042',
 'ING1043',
 'ING1045',
 'ING1056',
 'ING1057',
 'ING1058',
 'ING1100',
 'ING1200',
 'MAP1006',
 'MAP1007',
 'MAP1008',
 'STT1001',
 'GIA1082',
 'GMC1044',
 'GMC1030',
 'GMC1042',
 'ING1059',
 'GMC1019',
 'GMC1033',
 'GMC1038',
 'GMC1041']

# 2. Test run - try to scrape first course

In [17]:
driver.find_element_by_link_text(link_texts[0]).click()

In [18]:
page_soup = soup(driver.page_source, 'lxml')

In [19]:
course_title = page_soup.find("div", {"id": "zoneentetecontenu"}).find("h1", {"class": "titrepagecours"}).text.strip()
course_title

'GEI1085 - Outils pour la mécatronique'

In [20]:
course_code = course_title.split(" - ")[0]
course_code

'GEI1085'

In [21]:
course_name = course_title.split(" - ")[1]
course_name

'Outils pour la mécatronique'

In [22]:
course_desc = page_soup.find("div", {"class": "textedescription"}).find("div").text.strip()
course_desc

'Initiation aux systèmes électromécaniques possédant des composants électriques/électroniques : méthodes et outils de calcul des circuits électriques appliqués aux systèmes mécaniques. Introduction à l’analyse des modules de transfert d’énergie électrique dans les systèmes électromécaniques avec les composants semi-conducteurs. Introduction aux éléments d’interfaçage avec un automate programmable : photodiodes, phototransistors, optocoupleurs. Analyse des besoins de systèmes de transfert d’énergie pour les systèmes électromécaniques: décodage de plaques signalétiques de batteries, moteurs électriques, actionneurs pneumatiques et hydrauliques.'

In [23]:
driver.back()

# 3. Test run complete. Implement automation script to scrape all courses

In [24]:
course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title = page_soup.find("div", {"id": "zoneentetecontenu"}).find("h1", {"class": "titrepagecours"}).text.strip()
    course_code = course_title.split(" - ")[0]
    course_name = course_title.split(" - ")[1]
    course_desc = page_soup.find("div", {"class": "textedescription"}).find("div").text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  GEI1085 Outils pour la mécatronique 
 Initiation aux systèmes électromécaniques possédant des composants électriques/électroniques : méthodes et outils de calcul des circuits électriques appliqués aux systèmes mécaniques. Introduction à l’analyse des modules de transfert d’énergie électrique dans les systèmes électromécaniques avec les composants semi-conducteurs. Introduction aux éléments d’interfaçage avec un automate programmable : photodiodes, phototransistors, optocoupleurs. Analyse des besoins de systèmes de transfert d’énergie pour les systèmes électromécaniques: décodage de plaques signalétiques de batteries, moteurs électriques, actionneurs pneumatiques et hydrauliques.
1 courses scraped

Scraped  GEI1086 Instrumentation en mécatronique 
 Acquisition de connaissances sur les constituants d’une chaîne d’instrumentation d’un système mécatronique : modules d’acquisition et de conditionnement de signaux issus de montages mécaniques (amplification, atténuation, filtrage ac

Scraped  GMC1028 Procédés de mise en forme et de soudage 
 Compréhension des procédés de mise en forme primaire (métallurgie des poudres, fonderie, moulage par injection) et de la pertinence d'utilisation de l'un ou l'autre de ces procédés pour l'obtention de pièces brutes.

Métallurgie des poudres : fabrication des poudres, propriétés, compactage, frittage, conception des outillages, conception des pièces et applications, aspects économiques. Moulage : mécanismes impliqués dans la solidification, courbes de solidification, loi de Chvorinov, système de coulée et masselottes, procédés de moulage en moule permanent et non-permanent, conception des moules, considérations de conception des pièces à mouler. Soudage. Types de procédés, types de joints et positions de soudage, symboles de soudage, zone affectée thermiquement, soudabilité des métaux ferreux et non-ferreux. Normes ACNOR. Joints pré-qualifiés et qualification. Modèles de coûts de soudage. Laboratoires de soudage de divers types 

Scraped  ING1042 Dessin technique et DAO 
 Développer, chez l'étudiant en ingénierie, une dextérité manuelle et intellectuelle, des éléments indispensables à un langage et une expression graphique propres à ses fonctions futures; l'atteinte de ces objectifs est réalisée par le développement de l'esprit d'observation, du sens de la précision et de la capacité de s'exprimer graphiquement. Le cours vise également l'acquisition d'une connaissance générale des éléments indispensables à l'élaboration, au développement et à la mise en oeuvre d'un projet en ingénierie.

Acquérir les connaissances et les habiletés requises pour pouvoir utiliser le dessin technique comme moyen de communication dans les principaux champs d'activités en ingénierie. Assimiler les notions et techniques de base requises à la conception de dessins techniques assistée par ordinateur.
21 courses scraped

Scraped  ING1043 Matériaux de l'ingénieur 
 Acquérir la connaissance des caractéristiques générales physiques et des 

Scraped  MAP1007 Mathématiques appliquées II 
 Acquisition de notions fondamentales sur les fonctions à plusieurs variables, les courbes et les surfaces dans le plan et l'espace.  Applications du calcul différentiel et intégral à plusieurs variables ainsi que le calcul vectoriel.Eléments de géométrie analytique.  Courbes planes, tangentes et longueur d'arc, coordonnées polaires, intégrales en coordonnées polaires, équations polaires des coniques.  Vecteurs de dimension deux et trois, produit scalaire, produit vectoriel, droites et plans, surfaces.  Fonctions vectorielles et courbes dans l'espace, limites, dérivées et intégrales, cinématique, courbure, composantes tangentielle et normale de l'accélération, lois de Kepler.  Fonctions de plusieurs variables, limites et continuité, dérivées partielles, accroissement et différentielle, dérivation de fonctions composées, dérivées directionnelles, normales et plans tangents, valeurs extrêmes de fonctions à plusieurs variables, multiplicateurs

Scraped  GMC1019 Chauffage, ventilation et climatisation 
 Chauffage : pertes de chaleur, corps de chauffe, fournaise, systèmes de chauffage : à vapeur, à eau chaude, à air chaud; disposition et calcul de la tuyauterie, système à eau chaude à haute température, panneaux rayonnants, commandes automatiques.

Plomberie : eau froide et chaude domestique, étude et application des règlements de plomberie, systèmes de gicleurs automatiques.

Climatisation : réactions physiologiques des humains aux différentes ambiances, charge de climatisation, propriétés physiques de l'air, psychrométrie, procédés fondamentaux de conditionnement : chauffage, refroidissement, humidification, assèchement; distribution de l'air, canalisation de l'air à haute et basse vitesse, ventilateurs, filtration, régulateurs pour systèmes de climatisation, étude des différents systèmes installés dans les bâtiments.  Travaux pratiques qui consistent à calculer un système de climatisation pour un bâtiment.

Réfrigération : r

# 4. Inspect and write to CSV

In [26]:
course_codes

['GEI1085',
 'GEI1086',
 'GIA1047',
 'GIA1058',
 'GIA1077',
 'GMC1006',
 'GMC1016',
 'GMC1017',
 'GMC1024',
 'GMC1025',
 'GMC1028',
 'GMC1031',
 'GMC1032',
 'GMC1034',
 'GMC1035',
 'GMC1036',
 'GMC1037',
 'GMC1053',
 'ING1039',
 'ING1040',
 'ING1042',
 'ING1043',
 'ING1045',
 'ING1056',
 'ING1057',
 'ING1058',
 'ING1100',
 'ING1200',
 'MAP1006',
 'MAP1007',
 'MAP1008',
 'STT1001',
 'GIA1082',
 'GMC1044',
 'GMC1030',
 'GMC1042',
 'ING1059',
 'GMC1019',
 'GMC1033',
 'GMC1038',
 'GMC1041']

In [27]:
course_names

['Outils pour la mécatronique',
 'Instrumentation en mécatronique',
 'Analyse de rentabilité de projets I',
 'Sécurité et hygiène industrielles',
 'Conception de systèmes de fiabilité et de maintenance',
 'Mécanique des machines',
 'Elasticité et plasticité',
 'Design des éléments de machines',
 'Automatismes industriels',
 'Robotique industrielle',
 'Procédés de mise en forme et de soudage',
 'Asservissements et commande des systèmes mécaniques',
 'Conception et modélisation en ingénierie I',
 'Introduction à la méthode des éléments finis',
 "Méthodes numériques appliquées à l'ingénierie",
 'Procédés de coupe et de déformation',
 'Systèmes hydrauliques et pneumatiques',
 'Choix de projet de conception',
 'Statique et dynamique I',
 'Statique et dynamique II',
 'Dessin technique et DAO',
 "Matériaux de l'ingénieur",
 'Tolérancement et CAO',
 'Résistance des matériaux',
 'Thermodynamique appliquée I',
 "Phénomènes d'échanges",
 'Communication et méthodes de travail en ingénierie',
 "Pra

In [28]:
course_descs

['Initiation aux systèmes électromécaniques possédant des composants électriques/électroniques : méthodes et outils de calcul des circuits électriques appliqués aux systèmes mécaniques. Introduction à l’analyse des modules de transfert d’énergie électrique dans les systèmes électromécaniques avec les composants semi-conducteurs. Introduction aux éléments d’interfaçage avec un automate programmable : photodiodes, phototransistors, optocoupleurs. Analyse des besoins de systèmes de transfert d’énergie pour les systèmes électromécaniques: décodage de plaques signalétiques de batteries, moteurs électriques, actionneurs pneumatiques et hydrauliques.',
 'Acquisition de connaissances sur les constituants d’une chaîne d’instrumentation d’un système mécatronique : modules d’acquisition et de conditionnement de signaux issus de montages mécaniques (amplification, atténuation, filtrage actif et passif), modules de numérisation de signaux analogiques pour les applications d’automates programmables,

In [30]:
course_descs1 = [desc.split("Préalable 1 :")[0].replace("\n", " ").strip() for desc in course_descs]
course_descs1

['Initiation aux systèmes électromécaniques possédant des composants électriques/électroniques : méthodes et outils de calcul des circuits électriques appliqués aux systèmes mécaniques. Introduction à l’analyse des modules de transfert d’énergie électrique dans les systèmes électromécaniques avec les composants semi-conducteurs. Introduction aux éléments d’interfaçage avec un automate programmable : photodiodes, phototransistors, optocoupleurs. Analyse des besoins de systèmes de transfert d’énergie pour les systèmes électromécaniques: décodage de plaques signalétiques de batteries, moteurs électriques, actionneurs pneumatiques et hydrauliques.',
 'Acquisition de connaissances sur les constituants d’une chaîne d’instrumentation d’un système mécatronique : modules d’acquisition et de conditionnement de signaux issus de montages mécaniques (amplification, atténuation, filtrage actif et passif), modules de numérisation de signaux analogiques pour les applications d’automates programmables,

In [31]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs1
})

df

,Course Number,Course Name,Course Description
0,GEI1085,Outils pour la mécatronique,Initiation aux systèmes électromécaniques poss...
1,GEI1086,Instrumentation en mécatronique,Acquisition de connaissances sur les constitua...
2,GIA1047,Analyse de rentabilité de projets I,Situer les principaux éléments du contexte éco...
3,GIA1058,Sécurité et hygiène industrielles,Les objectifs principaux de ce cours sont d'in...
4,GIA1077,Conception de systèmes de fiabilité et de main...,Connaître la théorie de la fiabilité des équip...
5,GMC1006,Mécanique des machines,Acquisition des connaissances indispensables à...
6,GMC1016,Elasticité et plasticité,Acquisition de la capacité d'élaborer et de ré...
7,GMC1017,Design des éléments de machines,Acquisition des connaissances générales sur la...
8,GMC1024,Automatismes industriels,Ce cours vise l'acquisition des connaissances ...
9,GMC1025,Robotique industrielle,Ce cours de synthèse vise à intégrer un ensemb...


In [32]:
df.to_csv('UQTR_GenieMecanique_Core_and_Electives_Courses.csv', index = False)

In [33]:
driver.quit()